In [9]:
# Import pandas 
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns


# Read in white wine data 
white = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv", sep=';')

# Read in red wine data 
red = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=';')

In [10]:
# Add `type` column to `red` with value 1
red['type'] = 1

# Add `type` column to `white` with value 0
white['type'] = 0

# Append `white` to `red`
wines = red.append(white, ignore_index=True)

In [11]:
y = wines.quality
X = wines.drop(['quality', 'residual sugar', 'free sulfur dioxide', 'type'], axis=1)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=46, stratify=y)

# Define the scaler 
scaler = StandardScaler().fit(X_train)

# Scale the train set
X_train_scaled = scaler.transform(X_train)

# Scale the test set
X_test_scaled = scaler.transform(X_test)

In [12]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras import layers, optimizers, regularizers
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras import metrics

from keras.utils import plot_model
#from kt_utils import *
import keras.backend as K
from sklearn import preprocessing, model_selection 

In [13]:
def create_model():
    # create model
    model = Sequential()
    # layer 1
    model.add(Dense(9, input_dim=9, activation='relu', kernel_initializer='normal') )
    #layer 2
    model.add(Dense(50, activation='relu', kernel_initializer='normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    #layer 4
    model.add(Dense(1, activation='linear'))

    # Compile model
    model.compile(optimizer = 'adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasRegressor(build_fn=create_model, verbose=0)

# define the grid search parameters
batch_size = [100,150,200,250,300]
epochs = [5,10,20,40,60,80,100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_scaled, y_train)

In [14]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (abs(mean), stdev, param))

Best: -0.524971 using {'batch_size': 150, 'epochs': 100}
3.835788 (0.273399) with: {'batch_size': 100, 'epochs': 5}
0.590092 (0.015708) with: {'batch_size': 100, 'epochs': 10}
0.542204 (0.025895) with: {'batch_size': 100, 'epochs': 20}
0.537734 (0.017699) with: {'batch_size': 100, 'epochs': 40}
0.538312 (0.026126) with: {'batch_size': 100, 'epochs': 60}
0.532193 (0.026882) with: {'batch_size': 100, 'epochs': 80}
0.533091 (0.030598) with: {'batch_size': 100, 'epochs': 100}
12.296338 (0.571732) with: {'batch_size': 150, 'epochs': 5}
0.798112 (0.108152) with: {'batch_size': 150, 'epochs': 10}
0.571611 (0.035491) with: {'batch_size': 150, 'epochs': 20}
0.545635 (0.018262) with: {'batch_size': 150, 'epochs': 40}
0.546828 (0.030945) with: {'batch_size': 150, 'epochs': 60}
0.537868 (0.018149) with: {'batch_size': 150, 'epochs': 80}
0.524971 (0.030906) with: {'batch_size': 150, 'epochs': 100}
18.795101 (1.557046) with: {'batch_size': 200, 'epochs': 5}
3.192097 (0.397795) with: {'batch_size': 2

In [15]:
#Use scikit-learn to grid search the number of neurons
def create_model(neurons=1):
    # create model
    model = Sequential()
    # layer 1
    model.add(Dense(9, input_dim=9, activation='relu', kernel_initializer='normal'))
    #layer 2
    model.add(Dense(neurons, activation='relu', kernel_initializer='normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    #layer 3
    model.add(Dense(1, activation='linear'))
    
    # Compile model
    model.compile(optimizer = 'adam', loss='mean_squared_error', metrics=['accuracy'])
    return model


# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasRegressor(build_fn=create_model, epochs=100, batch_size=150, verbose=0)
# define the grid search parameters
neurons = [1, 5, 10, 20, 40, 60, 80, 100]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_scaled, y_train)

In [16]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.524911 using {'neurons': 60}
-4.104506 (0.221393) with: {'neurons': 1}
-1.200867 (0.052649) with: {'neurons': 5}
-0.692870 (0.085531) with: {'neurons': 10}
-0.607553 (0.081301) with: {'neurons': 20}
-0.553644 (0.041257) with: {'neurons': 40}
-0.524911 (0.027697) with: {'neurons': 60}
-0.526432 (0.026803) with: {'neurons': 80}
-0.527785 (0.025104) with: {'neurons': 100}


In [17]:
def create_model(hidden_layers=1):
    # create model
    model = Sequential()
    # layer 1
    model.add(Dense(9, input_dim=9, activation='relu',kernel_initializer='normal'))
    
    #hidden layers
    for i in range(hidden_layers):
        # Add one hidden layer
        model.add(Dense(60, activation='relu',  kernel_initializer='normal'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
    
    #layer 4
    model.add(Dense(1, activation='linear'))
    
    # Compile model
    model.compile(optimizer = 'adam', loss='mean_squared_error', metrics=['accuracy'])
    return model


# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasRegressor(build_fn=create_model, batch_size=150, verbose=0, epochs=100)
# define the grid search parameters
hidden_layers = [1,2,3,4]
param_grid = dict(hidden_layers = hidden_layers)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_scaled, y_train)


In [18]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.541322 using {'hidden_layers': 2}
-0.549916 (0.017436) with: {'hidden_layers': 1}
-0.541322 (0.022853) with: {'hidden_layers': 2}
-0.548761 (0.027697) with: {'hidden_layers': 3}
-0.553004 (0.023959) with: {'hidden_layers': 4}


In [20]:
# Use scikit-learn to grid search the activation function
def create_model(activation='relu'):
    # create model
    model = Sequential()
    # layer 1
    model.add(Dense(9, input_dim=9, activation='relu', kernel_initializer='normal'))
    #layer 2
    model.add(Dense(60, activation=activation,kernel_initializer='normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    #layer 3
    model.add(Dense(60, activation=activation,kernel_initializer='normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    #layer 4
    model.add(Dense(1, activation='linear'))
    
    # Compile model
    model.compile(optimizer = 'adam', loss='mean_squared_error', metrics=['accuracy'])
    return model


# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasRegressor(build_fn=create_model, epochs=100, batch_size=150, verbose=0)
# define the grid search parameters
activation = ['relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_scaled, y_train)

In [21]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.536695 using {'activation': 'linear'}
-0.538572 (0.025908) with: {'activation': 'relu'}
-0.555273 (0.023452) with: {'activation': 'tanh'}
-0.656335 (0.131770) with: {'activation': 'sigmoid'}
-0.631365 (0.115903) with: {'activation': 'hard_sigmoid'}
-0.536695 (0.025092) with: {'activation': 'linear'}


In [22]:
from keras.optimizers import Adam
def create_model(learn_rate=0.01):
    # create model
    model = Sequential()
    # layer 1
    model.add(Dense(9, input_dim=9, activation='relu', kernel_initializer='normal'))
    #layer 2
    model.add(Dense(60, activation='relu', kernel_initializer='normal'))
    #layer 3
    model.add(Dense(60, activation='relu',kernel_initializer='normal'))
    #layer 4
    model.add(Dense(1, activation='linear'))
    optimizer = Adam(lr=learn_rate)
    # Compile model
    model.compile(optimizer = optimizer, loss='mean_squared_error', metrics=['accuracy'])
    return model

import numpy as np
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasRegressor(build_fn=create_model, epochs=100, batch_size=150, verbose=0)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
param_grid = dict(learn_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_scaled, y_train)

In [23]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.530466 using {'learn_rate': 0.001}
-0.530466 (0.017739) with: {'learn_rate': 0.001}
-0.559180 (0.051520) with: {'learn_rate': 0.01}
-2.614763 (2.903250) with: {'learn_rate': 0.1}
-0.781056 (0.043106) with: {'learn_rate': 0.2}
-0.764899 (0.028628) with: {'learn_rate': 0.3}
